In [1]:
import json
import re

%load_ext autoreload
%autoreload 2

In [2]:
def form_element(idx, movie_name):
    return f"""
    <input type="checkbox" name="movie{str(idx)}" value={str(idx)}>
    <label for="movie{str(idx)}">{movie_name}</label><br>\n
    """

def get_all_inputs():
    form_element_str = ""
    for d in data["nodes"]:
        form_element_str += form_element(d["idx"], d["id"])
        
def clean_movie_titles(String):
    String = re.sub("TheThe", "The", String)
    String = re.sub("Hangover Part III, The Hangover Part III", "The Hangover Part III", String)
    String = re.sub("Last Exorcism Part II, The Last Exorcism Part II", "The Last Exorcism Part II", String)
    String = re.sub("Pretty One, The Pretty One", "The Pretty One", String)
    String = re.sub("Smurfs 2, The Smurfs 2", "The Smurfs 2", String)
    String = re.sub("Smurfs: The Legend of Smurfy Hollow, The Smurfs: The Legend of Smurfy Hollow", "The Smurfs: The Legend of Smurfy Hollow", String)
    String = re.sub("Catching Fire, The Hunger Games:The Hunger Games: Catching Fire", "The Hunger Games: Catching Fire", String)
    String = re.sub("Haunting in Connecticut 2: Ghosts of Georgia, The Haunting in Connecticut 2: Ghosts of Georgia", "The Haunting in Connecticut 2: Ghosts of Georgia", String)
    String = re.sub("Face of Love, The Face of Love", "The Face of Love", String)
    
    return String

In [3]:
with open('data.json', encoding='utf-8') as data_file:
    data = json.loads(data_file.read())

In [4]:
new_data_links = []
for d in data["links"]:
    if (d["source"] != d["target"]):
        d["source"] = clean_movie_titles(d["source"])
        d["target"] = clean_movie_titles(d["target"])
        new_data_links.append(d)

In [5]:
# ensure every link has a movie in node

wanted_indexes = []
for L in new_data_links:
    if L['source_idx'] not in wanted_indexes:
        wanted_indexes.append(L['source_idx'])
    if L['target_idx'] not in wanted_indexes:
        wanted_indexes.append(L['target_idx'])


new_data_nodes = []  
for n in data["nodes"]:
    
    if n["idx"] in wanted_indexes:
        new_n = {}
        new_n["idx"] = n["idx"]
        new_n["id"] = clean_movie_titles(n["id"])
        new_n["genre"] = n["genre"]
        new_n["description"] = n["description"]
        
        new_data_nodes.append(new_n)

In [6]:
new_json_data = {
    "nodes" : new_data_nodes,
    "links" : new_data_links
}

In [7]:
new_json_data

{'nodes': [{'idx': 4,
   'id': 'Alice in Wonderland',
   'genre': ['family', 'sciencefiction/fantasy'],
   'description': "Troubled by a strange recurring dream and mourning the loss of her father, 19-year-old Alice Kingsleigh attends a garden party at Lord Ascot's estate.  There, she is confronted by an unwanted marriage proposal to Lord Ascot's son, Hamish, and the stifling expectations of the society in which she lives.  Unsure of how to proceed, she pursues a rabbit wearing a blue waistcoat and accidentally falls into a large rabbit hole under a tree..."},
  {'idx': 1299,
   'id': 'Alice Through the Looking Glass',
   'genre': ['sciencefiction/fantasy'],
   'description': 'Chapter One – Looking-Glass House: Alice is playing with a white kitten (whom she calls "Snowdrop") and a black kitten (whom she calls "Kitty")—the offspring of Dinah, Alice\'s cat in Alice\'s Adventures in Wonderland—when she ponders what the world is like on the other side of a mirror\'s reflection.  Climbing u

In [8]:
with open('./data-v0.1.json', 'w') as outfile:
      json.dump(new_json_data, outfile)